In [1]:
%matplotlib inline
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
url = 'https://raw.githubusercontent.com/tomwhite/covid-19-uk-data/master/data/covid-19-cases-uk.csv'
df = pd.read_csv(url, error_bad_lines=False)


In [3]:
#format as date
df['Date']=pd.to_datetime(df.Date)
#sort by date
df.sort_values(by='Date')

#change '1 to 4' to 2
df.TotalCases = pd.to_numeric(df.TotalCases, errors='coerce').fillna(2).astype(np.int64)

#filter out awaiting clarification areas
df=df[df.Area!='awaiting clarification']
df=df[df.Area!='Awaiting confirmation']

In [4]:
#filter out outliers
df=df.drop(df[(df['Area']=='Sheffield')&(df['Date']==datetime.datetime(2020, 3, 17))].index)
df=df.drop(df[(df['Area']=='Derbyshire')&(df['Date']==datetime.datetime(2020, 3, 17))].index)

#drop To be confirmed and Resident outside Wales
df=df.drop(df[df['Area']=='To be confirmed'].index)
df=df.drop(df[df['Area']=='Resident outside Wales'].index)

In [5]:
#Just filter out Wales
wales=df[df['Country']=='Wales']

#read lookup csv
lookup = pd.read_csv("Unitary_Authority_to_Local_Health_Board_April_2019_Lookup_in_Wales.csv")

#Keep list of UAs
UAs=lookup['UA19NM'].values.tolist()

#Drop unnecessary columns
lookup=lookup.drop(columns=['UA19NM','FID'])

#Add in health boards
wales=wales.merge(lookup,left_on='AreaCode',right_on='UA19CD')
#Group by date and health board, summing as you go
wales2=wales.groupby(['Date','LHB19NM','LHB19CD'], as_index=False).sum()

#add it back into the dataframe
for index,row in wales2.iterrows():
#     print(row.Date, row.LHB19NM, row.LHB19CD,index)
    df=df.append({'Date':row.Date,'Country':'Wales','AreaCode':row.LHB19CD,'Area':row.LHB19NM,'TotalCases':index}, ignore_index=True)

# Keep Powys as it's a one UA health board
UAs.remove('Powys')
#now get rid of the old UAs
df=df.loc[~df['Area'].isin(UAs)]

# resort by date
df.sort_values('Date')


Date   Country   AreaCode                Area  TotalCases
0    2020-03-01  Scotland  S08000015  Ayrshire and Arran           0
13   2020-03-01  Scotland  S08000028       Western Isles           0
12   2020-03-01  Scotland  S08000030             Tayside           0
11   2020-03-01  Scotland  S08000026            Shetland           0
10   2020-03-01  Scotland  S08000025              Orkney           0
...         ...       ...        ...                 ...         ...
7254 2020-04-15   England  E08000004              Oldham         370
7253 2020-04-15   England  E10000024     Nottinghamshire         693
7252 2020-04-15   England  E06000018          Nottingham         334
7258 2020-04-15   England  E06000044          Portsmouth         193
7198 2020-04-15   England  E06000059              Dorset         209

[7151 rows x 5 columns]

In [6]:
#create dictionary for areas with more than 10 cases
areaWithMoreThan10={}


for index, row in df.iterrows():
    if row.TotalCases>10:
#         print(row.Area,row.Date)
        if row.Area not in areaWithMoreThan10.keys():
            areaWithMoreThan10[row.Area] = {'Area':row.Area,"Code":row.AreaCode,"firstDate":row.Date}


In [7]:
#Work out time since 10 cases
theseAreas=[]

for index, row in df.iterrows():
    if row.Area in areaWithMoreThan10.keys():
#         print(row.Date-areaWithMoreThan10[row.Area]['firstDate'])
        if row.Date-areaWithMoreThan10[row.Area]['firstDate'] >= datetime.timedelta(days = 0):
            theseAreas.append(row.Date-areaWithMoreThan10[row.Area]['firstDate'])
        else:
            theseAreas.append(None)
    else:
        theseAreas.append(None)

df['timeSince10']=theseAreas        

In [8]:
filterArea=df[~pd.isnull(df.timeSince10)]
#just get data where cases have timeSince10

In [9]:
lines=filterArea.pivot(index='Area',values='TotalCases',columns='timeSince10').transpose()

In [10]:
#lines.plot(figsize=[12,13],yscale='log')
#ah balls, just export it and do it in datawrapper

In [11]:
#save as .csv
lines.to_csv('jbm '+str(datetime.date.today())+'.csv', index = True)